
pip intall openpyxl

import pandas as pd
import numpy as np
import os
from IPython.display import clear_output

In [79]:
import pandas as pd
import numpy as np
import os


# Tipos do espd0068
espd068Type = {
    'Repres': 'int64',                 
    'Nome Repres': 'object',          
    'Estab': 'int64',                  
    'Nr Nota Fiscal': 'int64',         
    'Dt Emis Nota': 'str',          
    'Cod Cliente': 'int64',           
    'Nome Cliente': 'object',          
    'Pedido': 'int64',                 
    'Cod Item': 'int64',               
    'Desc Item': 'object',            
    'Qtde Faturada': 'int64',         
    'Valor Unit Item': 'object',       
    'Valor Total Item': 'object',      
    'Qtd Nao Atendida': 'int64',       
    'Valor Nao Atendido': 'object'
}

# Tipos do espd0028
espd028Type = {
    'Data_Implant': 'str',
    'Nr_Pedido': 'int64',
    'Nr_Cliente': 'int64',
    'Nome_Abrev': 'object',
    'Nome_Repre': 'object',
    'Familia': 'object', # Este campo está em um lugar incorreto no seu arquivo
    'Codigo_Item': 'int64',
    'Descricao': 'object',
    'Qt_Pedido': 'object',
    'R_Unit': 'object',
    'Qt_Aloc': 'int64',
    'Vlr_Total': 'object',
    'Est_Disp': 'object',
    'Situacao': 'object',
    'Entrega': 'str',
    'MesAno': 'int64',
    'Lista_Presente': 'int64',
    'Observacao': 'object',
    # As colunas abaixo parecem ser lixo e precisam ser ignoradas
    'Lixo1': 'object',
    'Lixo2': 'object',
    'Lixo3': 'object',
    'Lixo4': 'object',
    'Lixo5': 'object',
    'Lixo6': 'object',
    'Lixo7': 'object',
}

In [80]:
import pandas as pd


nomes_colunas = [
    'Data_Implant', 'Nr_Pedido', 'Nr_Cliente', 'Nome_Abrev', 'Nome_Repre',
    'Familia', 'Codigo_Item', 'Descricao', 'Qt_Pedido', 'R_Unit',
    'Qt_Aloc', 'Vlr_Total', 'Est_Disp', 'Situacao', 'Entrega',
    'MesAno', 'Lista_Presente', 'Observacao',
    'Coluna19', 'Coluna20', 'Coluna21', 'Coluna22', 'Coluna23', 'Coluna24', 'Coluna25'
]

espd028 = pd.read_csv(
    'espd0028.txt',
    sep='@',
    encoding='ISO-8859-1',
    header=None,
    names=nomes_colunas,
    skiprows=1,  
    dtype=espd028Type
)


colunas_para_remover = [
    'Coluna19', 'Coluna20', 'Coluna21', 'Coluna22', 'Coluna23', 'Coluna24', 'Coluna25'
]
espd028 = espd028.drop(columns=colunas_para_remover)


In [81]:
espd068_path = 'espd0068.txt'

espd068 = pd.read_csv(espd068_path, sep='@', quotechar='"', encoding='ISO-8859-1',dtype=espd068Type)


espd068['Dt Emis Nota'] = pd.to_datetime(espd068['Dt Emis Nota'], format='%d/%m/%y')
espd028['Data_Implant'] = pd.to_datetime(espd028['Data_Implant'], format='%d/%m/%y')
espd028 = espd028.dropna(axis=1, how='all')

espd028['Qt_Pedido'] = espd028['Qt_Pedido'].str.replace(',', '.', regex=False)

espd028['Est_Disp'] = espd028['Est_Disp'].str.replace(',', '.', regex=False).astype(float).round().astype('Int64')



In [82]:
#pedidos_validados =  [3280713, 3281004, 3280910, 3280706, 3280670, 3280049, 3279972, 3279970, 3279830, 3279827, 3279794, 3279684, 3279139, 3279138, 3279112, 3279067, 3278961, 3278849, 3278098, 3278021, 3277955, 3277923, 3277922, 3277911, 3277909, 3276941, 3277572, 3277458, 3277321, 3277320, 3277290, 3277163, 3277141, 3277113, 3277097, 3277091, 3277083, 3277050, 3277030, 3277024, 3277019, 3276995, 3276994, 3276993, 3276965, 3276963, 3276962, 3276960, 3276945, 3276944, 3276346, 3275299, 3274550, 3274448, 3274447, 3274423, 3274401, 3274412, 3274156, 3273706, 3272513, 3268221, 3268162, 3266131, 3266101, 3265952, 3264897, 3264858, 3264825, 3264857, 3263872, 3263811, 3261910, 3261703, 3261909, 3261160, 3259768, 3259340, 3259583, 3258742, 3256359, 3255807, 3255555, 3254971, 3254336, 3254180, 3254066, 3254063, 3252425, 3251764, 3251220, 3251072, 3250768, 3248435, 3248137, 3248110, 3248077, 3247907, 3247451, 3247325, 3246848, 3246733, 3246655, 3245862, 3244223, 3244222, 3243732, 3243723, 3243287, 3243276, 3242329, 3242251, 3241817, 3241708, 3241674, 3241673, 3241669, 3241668, 3241667, 3241666, 3241665, 3241660, 3241659, 3241635, 3241621, 3241619, 3241617, 3241359, 3241317, 3240804, 3240790, 3240769, 3240768, 3240765, 3240733, 3240732, 3240696, 3240694, 3240898, 3240897, 3240297, 3240102, 3240416, 3239741, 3239639, 3239589, 3239580, 3239486, 3239445, 3239407, 3239401, 3239399, 3239374, 3239371, 3239369, 3239367, 3239366, 3239345, 3239301, 3239291, 3239578, 3239124, 3239123, 3239121, 3239090, 3239088, 3239045, 3239010, 3238974, 3238945, 3238944, 3238923, 3238921, 3238884, 3238866, 3230194, 3220965, 3219721, 3219720, 3215194]
#espd028 = espd028[espd028['Nr.Pedido'].isin(pedidos_validados)]

In [83]:

itens_codigo_especifico = espd028[espd028['Codigo_Item'] == 4111160404]

itens_codigo_especifico = itens_codigo_especifico[['Codigo_Item', 'Est_Disp']]



####### Consolidando os dados no espd068 para cada combinação de Pedido e Cod Item

In [84]:


##espd028.to_excel('espd0028 base.xlsx',index=False)
espd068_consolidado = espd068.groupby(['Pedido', 'Cod Item'], as_index=False)['Qtde Faturada'].sum()


espd028 = espd028.merge(
    espd068_consolidado,
    left_on=['Nr_Pedido', 'Codigo_Item'],
    right_on=['Pedido', 'Cod Item'],
    how='left'
)


espd028 = espd028.rename(columns={'Qtde Faturada': 'Qtde_Faturada'})


espd028 = espd028.drop(columns=['Pedido', 'Cod Item'])


espd028['Qtde_Faturada'] = espd028['Qtde_Faturada'].fillna(0).astype('Int64')



In [85]:
qtde_faturada_maior_que_zero = espd028[espd028['Qtde_Faturada'] > 0]


In [86]:

espd028['Qt_Aloc'] = espd028['Qt_Aloc'] - espd028['Qtde_Faturada']

In [87]:
qtde_faturada_maior_que_zero = espd028[espd028['Qtde_Faturada'] > 0]

In [88]:
import numpy as np


espd028['Qt_Pedido'] = np.where(
    espd028['Situacao'] == 'Total', 
    0, 
    np.where(
        espd028['Qt_Pedido'].fillna(0).astype(float) - espd028['Qt_Aloc'].fillna(0).astype(float) < 0, 
        0, 
        espd028['Qt_Pedido'].fillna(0).astype(float) - espd028['Qt_Aloc'].fillna(0).astype(float)
    )
)

espd028['Qt_Pedido'] = espd028['Qt_Pedido'].astype(float).round().astype('Int64')



In [89]:
transito = pd.read_excel('transito.xlsx', header=0)

text_columns = transito.columns[transito.columns.map(lambda x: isinstance(x, str))]

espd028 = espd028.merge(
    transito,
    left_on='Codigo_Item',
    right_on='Cod',
    how='left'
)
for column in text_columns:
    espd028[column] = espd028[column].fillna(0).astype(int)
espd028 = espd028.drop(columns=['Cod'])

In [90]:
text_columns = transito.columns.tolist()
if 'Cod' in text_columns:
    text_columns.remove('Cod')
text_columns.insert(0, 'Est_Disp')

In [91]:
estoque_inicial = espd028[['Codigo_Item'] + text_columns].copy()


estoque_inicial = estoque_inicial.drop_duplicates(subset='Codigo_Item')


estoque_inicial[text_columns] = estoque_inicial[text_columns].fillna(0).astype(int)


In [92]:
import random


espd028.to_excel('espd0028 final.xlsx',index=False)

espd028['Valor Item'] = espd028['R_Unit'].apply(lambda x: float(x.replace(',', '.'))) * espd028['Qt_Pedido']


pedidos_totais = espd028.groupby('Nr_Pedido', as_index=False)['Valor Item'].sum()
pedidos_totais = pedidos_totais.rename(columns={'Valor Item': 'Valor Total Pedido'})


espd028 = espd028.merge(pedidos_totais, on='Nr_Pedido', how='left')


pedidos = espd028['Nr_Pedido'].unique()


pedidos_maior_para_menor = list(pedidos_totais.sort_values(by='Valor Total Pedido', ascending=False)['Nr_Pedido'])

pedidos_menor_para_maior = list(pedidos_totais.sort_values(by='Valor Total Pedido', ascending=True)['Nr_Pedido'])

pedidos_aleatorio1 = list(pedidos)
pedidos_aleatorio2 = list(pedidos)
random.shuffle(pedidos_aleatorio1)
random.shuffle(pedidos_aleatorio2)

estrategias = {
    'Maior para Menor': pedidos_maior_para_menor,
    'Menor para Maior': pedidos_menor_para_maior,
    'Aleatório 1': pedidos_aleatorio1,
    'Aleatório 2': pedidos_aleatorio2
}

def simular_faturamento(pedidos_ordem, espd028, estoque_inicial, text_columns):
    faturado_total = []

    estoque_sim = estoque_inicial.copy(deep=True)
    ordem_faturamento = 0

    for pedido in pedidos_ordem:
        ordem_faturamento += 1  
        itens_pedido = espd028[espd028['Nr_Pedido'] == pedido].copy()

       
        for estoque_col in text_columns:
         
            for idx, item in itens_pedido.iterrows():
                qt_pedido = item['Qt_Pedido']
                if qt_pedido <= 0:
                    continue 

                cliente = item['Nr_Cliente']
                familia = item['Familia']
                codigo_item = item['Codigo_Item']
                valor_unit = float(item['R_Unit'].replace(',', '.'))

               
                if codigo_item not in estoque_sim['Codigo_Item'].values:
                    continue

              
                estoque_item = estoque_sim[estoque_sim['Codigo_Item'] == codigo_item].iloc[0]
                estoque_atual = estoque_item[estoque_col]

                if estoque_atual > 0:
                    qt_faturada = min(qt_pedido, estoque_atual)
                    valor_faturado = qt_faturada * valor_unit

                    faturado_total.append({
                        'Nr_Pedido': pedido,
                        'Nr_Cliente': cliente,
                        'Familia': familia,
                        'Codigo_Item': codigo_item,
                        'Qtde Faturada': qt_faturada,
                        'Valor Faturado': valor_faturado,
                        'Estoque Faturado': estoque_col,
                        'Ordem Faturamento Pedido': ordem_faturamento,
                    })

                  
                    itens_pedido.loc[idx, 'Qt_Pedido'] -= qt_faturada
                    estoque_sim.loc[estoque_sim['Codigo_Item'] == codigo_item, estoque_col] -= qt_faturada

    
    return pd.DataFrame(faturado_total)

with pd.ExcelWriter('Faturado_Total_Estrategias.xlsx') as writer:
    for nome_estrategia, pedidos_ordem in estrategias.items():
        resultado = simular_faturamento(pedidos_ordem, espd028, estoque_inicial, text_columns)
        resultado.to_excel(writer, sheet_name=nome_estrategia, index=False)

print('Simulação de faturamento por pedido concluída. Arquivo "Faturado_Total_Estrategias.xlsx" gerado.')

Simulação de faturamento por pedido concluída. Arquivo "Faturado_Total_Estrategias.xlsx" gerado.
